In [1]:
from pyEDM import *
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
import xarray as xr
from mpl_toolkits.axes_grid1 import make_axes_locatable
import fiona
import rasterio.mask
from descartes import PolygonPatch
from rasterio.plot import show
from tqdm import tqdm
import datetime
import random
from sklearn.preprocessing import StandardScaler
import scipy.stats

In [2]:
precip_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/CHIRPS/Anomaly/'
temp_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/LST/Anomaly/'
ndvi_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly/'

In [28]:
data_volumes = '/home/rgreen/DroughtEDM/Data/data_volumes/'

In [3]:
def generate_dekads(start, end):
    '''
    Generates list of timestamps of dekads between two dates
    
    Parameters
    ----------
    start : pd.datetime object
        Start date of list
    end : pd.datetime object
        end date of list
    '''
    
    dtrange = pd.date_range(start, end)
    days = list(range(len(dtrange))) #length of dtrange
    daysDF = pd.DataFrame({'Days': days}, index=dtrange)
    d = daysDF.index.day - np.clip((daysDF.index.day-1) // 10, 0, 2)*10 - 1 
    dekaddates = daysDF.index.values - np.array(d, dtype="timedelta64[D]")
    dekads = daysDF.groupby(dekaddates).mean()
    dekads = dekads.index
    
    return dekads

In [4]:
start = pd.datetime(2002,7,1)
end = pd.datetime(2019,4,30)

dekads = generate_dekads(start, end)

In [5]:
def pixelwise_ts_table(in_dir, keyword):
    
    '''
    This function searches through the file directory and creates a dataframe of pixel values over time from rasters
    based on a given keyword describing the environmental variable of interest to generate a raster time series stack 
    
   In the output dataframe, columns are each pixel extracted from the raster (read squentially into a list)
   and rows are the same pixel over time (dekads going down)
    
    Args:
    - in_dir: path to the input directory
    -keyword: string that is unique to the environmental variable
        options: ['precip', 'temp', 'ndvi']
    '''
    
    
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains(keyword)
    files = files[tifs]
        
    pixelwise_TS = []
    
    for filename in tqdm(files): 
        
        
            open_file = xr.open_rasterio(in_dir+filename).sel(band=1)
            array = open_file.values
            
            pixel_list = array.ravel().tolist()
            
            pixelwise_TS.append(pixel_list)

    return pd.DataFrame(pixelwise_TS)

    

In [27]:
precip_table_anom = pixelwise_ts_table(precip_anom_dir, 'precip')
temp_table_anom = pixelwise_ts_table(temp_anom_dir, 'temp')
ndvi_table_anom = pixelwise_ts_table(ndvi_anom_dir, 'ndvi')

100%|██████████| 606/606 [00:09<00:00, 61.82it/s]


In [29]:
precip_table_anom.to_csv(data_volumes + 'precip_table_anom.csv')

In [30]:
temp_table_anom.to_csv(data_volumes + 'temp_table_anom.csv')

In [31]:
ndvi_table_anom.to_csv(data_volumes + 'ndvi_table_anom.csv')

In [14]:
ndvi_table_anom.insert(0, 'Time', dekads)

In [21]:
ndvi_table_anom.insert(0, 'Year',ndvi_table_anom['Time'].dt.year)


In [25]:
ndvi_table_anom.insert(0, 'Dekad',list(range(1,19)) + list(range(1,37))*16 + list(range(1,13)))


In [26]:
ndvi_table_anom

,Dekad,Year,Time,0,1,2,3,4,5,6,...,392932,392933,392934,392935,392936,392937,392938,392939,392940,392941
0,1,2002,2002-07-01,1.101671,1.267218,0.938137,1.100588,1.324199,1.315054,1.165266,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2002,2002-07-11,0.981070,1.093294,0.873698,1.124495,1.246212,1.232029,1.013628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2002,2002-07-21,0.620074,0.497290,0.323125,0.514614,1.014546,0.785064,0.773919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2002,2002-08-01,1.039260,0.837557,0.609005,0.773013,1.242413,1.024316,1.137690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2002,2002-08-11,0.971371,0.751863,0.467435,0.481555,1.118889,0.952852,1.045681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,8,2019,2019-03-11,0.208073,0.026296,-0.284296,0.197938,-0.367827,-0.628079,-0.157488,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,9,2019,2019-03-21,0.522922,0.110877,-0.002413,0.636307,-0.054164,-0.157637,0.066254,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,10,2019,2019-04-01,0.331863,-0.081993,-0.378869,-0.353615,-0.445762,-0.467402,-0.187034,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,11,2019,2019-04-11,0.306670,-0.136034,-0.296636,0.203851,-0.584455,-0.485288,-0.248363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
big_array = np.array()
for d in dekads:
    for y in years:
        for p in pixels:
            big_array[y,d,p] = ndvi_table_anom.loc(df['year'] == y and df['decad'] == d)[p]